# Давайте немного повторим с прошлых занятий

Что такое reduce и accumulate?<br>
Принцип работы словаря (хеширование ключа)?

 А еще, спасибо Роман (занятие потока 2024), прислал интересную формулировки задачки и решение с бегущими суммами с ЕГЭ:

 На прошлом занятии мы использовали словарь с позициями, тут используется массив со всеми возможными значениями k для хранения вспомогательной информации.

In [ ]:
# нужно найти непрерывную последовательность элементов кратную k, с наибольшей суммой и наименьшей длинной
f = list(map(int, input().split()))
k = int(input())
summa = 0
ost = [None] * k
ost[0] = [0, 0]
total_summa = 0
total_length = 0
# создаем переменные в которых содержится текущая сумма, итоговая сумма и итоговая длина

for i in range(len(f)):
    summa += f[i]
    r = summa % k
    if ost[r] is not None: # если до этого момента есть сумма с таким остатком нужно сравнить ее сумму и длину
        if summa - ost[r][0] > total_summa:
            total_summa = summa - ost[r][0]
            total_length = i - ost[r][1]
        elif summa - ost[r][0] == total_summa:
            if i - ost[r][1] < total_length:
                total_length = i - ost[r][1]
        if summa < ost[r][0]: # возможно, эта цепочка лучше страрой (меньше сумма / длиннее)
            ost[r] = [summa, i]
        elif summa == ost[r][0]:
            if i > ost[r][1]: # если нам нужна самая короткая цепочка, то отсекать от начала надо самую длинную
                ost[r][1] = i
    else: # если такого остатка раньше не было, то просто записать
        ost[r] = [summa, i]
print(total_length, total_summa)

# Продолжаем про декораторы

## Замена исходной функции

Функция, возвращаемая декоратором, заменяет декорируемую

In [ ]:
def deco(func):
    func() # можно к примеру ее выполнить,
    def inner():
        print('running inner()')
    return inner

In [ ]:
def target():
    print('running target()')

In [ ]:
@deco
def target():
    print('running target()')

running target()


In [ ]:
target()

running inner()


In [ ]:
target  # now reference to inner

<function __main__.deco.<locals>.inner()>

## Running at import time

Key feature декораторов в том, что они применяются сразу после объявления декорируемой функции во время загрузки модуля в Python

In [ ]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def run():
    print('running run()')
    print('registry', registry)
    f1()
    f2()
    f3()

running register <function f1 at 0x7fe3a688e0c0>
running register <function f2 at 0x7fe3a688e200>


In [ ]:
run()

running run()
registry [<function f1 at 0x7f353010fa30>, <function f2 at 0x7f35317560e0>]
running f1()
running f2()
running f3()


## Декоратор-логгер

### logging

 Модуль `logging` содержит набор функций для логирования различного поведения в вашей программе. Например, для понимания, что ваша программа работает как ожидается, или там идет что-то не так

Есть разные уровни логирования: debug, info, warning, error, critical

In [ ]:
import logging

In [ ]:
logging.info('Working as expected')
logging.warning('Smth might go wrong, check this!')

Уровень логгирования по умолчания Warning, info не пишется, но можно переконфигурировать

Можно логировать в файл (надо начать новую сессию в Python)

In [ ]:
import os
import logging

os.makedirs('data', exist_ok=True)
log_filename = os.path.join('data', 'example.log')
log_level = logging.DEBUG

# Есть конфигурация по умолчанию на colab, и ее нужно убрать, что бы конфигурация применилась
# Тоже бывает проблоема с юпитер ноутбуком
for handler in logging.root.handlers[:]:
    print(handler)
    logging.root.removeHandler(handler)

print("задаем конфигурацию")

logging.basicConfig(
    filename=log_filename,
    encoding='utf-8',
    level=log_level,
    format='%(asctime)s %(message)s'  # add time
)

for handler in logging.root.handlers[:]:
    print(handler)

logging.debug('One very thorough message to save in our log')
logging.info('Just some info, also in log')
logging.warning('Something might be wrong')
logging.error('Logging our failure')

<StreamHandler stderr (NOTSET)>
задаем конфигурацию
<FileHandler /content/data/example.log (NOTSET)>


In [ ]:
! cat data/example.log

2024-10-09 11:58:01,267 One very thorough message to save in our log
2024-10-09 11:58:01,267 Just some info, also in log
2024-10-09 11:58:01,268 Something might be wrong
2024-10-09 11:58:01,268 Logging our failure


https://docs.python.org/3/library/logging.handlers.html
Так же много разных интеграций, можно настроить ротацию лог сообщений, или отправку на уделнный сервис, к примеру в Elasticsearch, особенно удобно если у нас много рабочих нод, можно централизованно смотреть логи.

# Декоратор с параметрами
Он сначала вызывается для подготовки, с параметрами из декоратора

In [ ]:
def decorator_with_params(a):
    print(f"Decorator with params called {a}")

    def actual_decorator(func):
        print("Actual decorator called")
        return func

    return actual_decorator

@decorator_with_params(a = 10)
def my_function():
    print(f"my_function()")

Decorator with params called 10
Actual decorator called


In [ ]:
my_function()

my_function()


### Напишем log-декоратор

In [ ]:
import logging
import sys

from functools import wraps, partial

In [ ]:
logger = logging.getLogger(name='log')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(stream=sys.stdout)) # каждое выполнение добавляет логгер, предидущий остается, и можно увидеть
# после повторного выполнения этого кода более одного сообщения логгирования

In [ ]:
for handler in logging.root.handlers[:]:
    print(handler)

<FileHandler /content/data/example.log (NOTSET)>


https://docs.python.org/3/howto/logging.html#logging-flow

Можно почитать подробнее про иерархию логгирования

In [ ]:
def logged(func=None,
           *,
           log_level=logging.INFO,
           log_name=None,
           log_message=None):
    print(func == None)
    if func is None:
        return partial(logged, log_level=log_level, log_name=log_name, log_message=log_message)

    log_name = log_name if log_name else func.__name__
    log_message = log_message if log_message else func.__name__

    logger = logging.getLogger(log_name)
    logger.propagate = False

    @wraps(func)
    def wrapper(*args, **kwargs):
        logger.log(log_level, log_message)
        return func(*args, **kwargs)

    return wrapper

In [ ]:
@logged(log_name='log')
def add(x, y):
    return x + y

True
False


In [ ]:
add(1, 2)

3

Протестируем с декоратором с прошлых семинаров

In [ ]:
import time

def clock(func):
    @wraps(func)
    def clocked(*args):
        t_start = time.time()
        res = func(*args)
        total_time = time.time() - t_start
        print(f'{func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res
    return clocked

In [ ]:
@logged(log_name='log')
@clock
def add(x, y):
    time.sleep(2)
    return x + y

True
False


In [ ]:
add(10, 10**10)

add((10, 10000000000)) -> 10000000010 executed in 2.00s


10000000010

## Декоратор -- assert типов

Мы уже говорили про type hints, а также про то, что никаких ошибок без использования сторонних модулей некорректные типы нам не дадут. Давайте реализуем логику проверки типов с помощью декоратора.

Типы могут быть заданы как стандартными названиями, так и с использованыем модуля typing. Для Any пропускайте проверку

In [ ]:
from typing import Any

In [ ]:
def func(a: int, b: str) -> str:
    return a * b

https://typeguard.readthedocs.io/en/stable/userguide.html

In [ ]:
func('2', 'a')

TypeError: can't multiply sequence by non-int of type 'str'

Стоит выбросить ValueError, если есть несовпадение между реальным и ожидаемым типом у какой-то из переданных переменных или у возвращаемого значения

# OOP

In Python, object-oriented Programming (OOPs) is a programming paradigm that uses objects and classes in programming. It aims to implement real-world entities like inheritance, polymorphisms, encapsulation, etc. in the programming. The main concept of OOPs is to bind the data and the functions that work on that together as a single unit so that no other part of the code can access this data.

Язык может быть ориентирован на объекты, но программу, построенную по принципам ООП, можно написать и в языках, где нет как таковых классов/объектов. Например, на C, или на Go.

- **Абстракция**.

Выделение значимой информации и использование наиболее важных характеристик объекта.
Абстракция от общего к частному. Класс -> экземпляр класса, Базовый класс -> производный класс.

Выделяем общее из частного. Притом то общее, с которым мы хотим работать.

Пример абстракции. Если нам нужно, чтобы утка крякала и плавала, то все то, что крякает и плавает для нас будет уткой. Мы отбросили все частное, которое нам не нужно.

Можно сравнить с моделирование более сложного обьекта реальной жизни для экспериментов.

Также абстракция бывает горизонтальной. Когда мы моделируем нечто с помощью массива (множество, дерево и пр.). С помощью одного и того же инструмента можем определять разное. Другой пример через модульность.

Как это относится к ООП?


- **Инкапсуляция**

Логическое объединение данных и функций для работы с ними. Инструмент, который позволяет разделить действия, необходмые для разного уровня абстракций.

Минимизация необходимого информационного пространства, за счет создания иерархии пространств имен (размещаем в объектке и его описание, и методы, которые с этим объектом работают). Нотация доступа к методам объекта через точку. Методы, поля и прочее и прочее и прочее. Посмотреть методы можно с помощью команды `dir()`.

Следует заметить, что также часто используется также "сокрытие" определенных полей. Когда мы говорим, что внутри объекта есть еще поля, но мы вам их не покажем. В некоторых языках есть, например,


- **Наследование**

Повторное использование свойств объектов с описанием различий.

Хотим, когда мы используем очередной класс объектов, мы могли сказать "А можно сделать вот так же, как тут, только в некоторых пунктах мы делаем изменений". Повторное использование кода.


- **Полиморфизм**

Предоставление одинаковых средств взаимодействия с объектами разной природы. Возможность для одного и того же кода обрабатывать данные разных типов. Duck typing в Python -- это из коробки


Класс можно использовать как контейнер. Мы можем складывать в него дополнительные поля. Можно делать свои "пространства имен", и в них складывать какие-то методы. Например, можно сделать некоторый класс и объявить в нем свою функцию минимума. Или определить какое-то поле, например, value. И тогда value будет определена только в области видимости нашего класса

In [ ]:
# псевдокод пример
class Iterable:
  def next():
    pass

class Array(Iterable):
  def __init__(self, data):
    self.i = 0
    self.data = data

  def next():
    if self.i < len(self.data):
      res = self.data[self.i]
      self.i += 1
      return res
    else:
      raise StopIteration


class List(Iterable):
  def __init__(self, data):
    self.data = data
    self.head = data.firt

  def next():
    if self.head is None:
      raise StopIteration
    res = self.head.value
    self.head = self.head.next
    return res

arr = Array([1, 2, 3])
arr2 = Array([1, 2, 3, 4])
_list = List([1, 2, 3, 4])

def iter(iterable):
  while val := iterable.next() != None:
    print(val)

for element in [arr, arr2, _list]:
  iter(element)


**Class**
A class is a collection of objects. A class contains the blueprints or the prototype from which the objects are being created. It is a logical entity that contains some attributes and methods.

To understand the need for creating a class let’s consider an example, let’s say you wanted to track the number of dogs that may have different attributes like breed, age. If a list is used, the first element could be the dog’s breed while the second element could represent its age. Let’s suppose there are 100 different dogs, then how would you know which element is supposed to be which? What if you wanted to add other properties to these dogs? This lacks organization and it’s the exact need for classes.

Some points on Python class:  

Classes are created by keyword class.
Attributes are the variables that belong to a class.
Attributes are always public and can be accessed using the dot (.) operator. Eg.: Myclass.Myattribute

**Objects**
The object is an entity that has a state and behavior associated with it. It may be any real-world object like a mouse, keyboard, chair, table, pen, etc. Integers, strings, floating-point numbers, even arrays, and dictionaries, are all objects. More specifically, any single integer or any single string is an object. The number 12 is an object, the string “Hello, world” is an object, a list is an object that can hold other objects, and so on. You’ve been using objects all along and may not even realize it.

*An object consists of :*

- State: It is represented by the attributes of an object. It also reflects the properties of an object.

- Behavior: It is represented by the methods of an object. It also reflects the response of an object to other objects.

- Identity: It gives a unique name to an object and enables one object to interact with other objects.

Давай попробуем сравнить ООП и функциональное программирование?

## Обработка исключений

https://docs.python.org/3/c-api/exceptions.html


In [ ]:
a = '1234'

try:
    a[0] = '10'
except:
    print('impossible to modify object')

impossible to modify object


In [ ]:
a = '1234'

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')

impossible to modify object


In [5]:
a = '1234'

try:
    a[0] = '10'
except BaseException as e:
    print(f'impossible to modify object: {help(e)}')
    print('impossible to modify object')
finally:
    print('run finally in any case')

Help on TypeError object:

class TypeError(Exception)
 |  Inappropriate argument type.
 |
 |  Method resolution order:
 |      TypeError
 |      Exception
 |      BaseException
 |      object
 |
 |  Methods defined here:
 |
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |
 |  __new__(*args, **kwargs)
 |      Create and return a new object.  See help(type) for accurate signature.
 |
 |  ----------------------------------------------------------------------
 |  Methods inherited from BaseException:
 |
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |
 |  __reduce__(...)
 |      Helper for pickle.
 |
 |  __repr__(self, /)
 |      Return repr(self).
 |
 |  __setstate__(...)
 |
 |  __str__(self, /)
 |      Return str(self).
 |
 |  add_note(...)
 |      Exception.add_note(note) --
 |      add a 

In [ ]:
a = [1, 2]

try:
    a[0] = '10'
except BaseException:
    print('impossible to modify object')
finally:
    print('run finally in any case')

run finally in any case


https://docs.python.org/3/library/exceptions.html

In [ ]:
issubclass(TypeError, BaseException), issubclass(ValueError, BaseException)

(True, True)

In [ ]:
TypeError.mro(), ValueError.mro()  # смотрим цепочку наследования

([TypeError, Exception, BaseException, object],
 [ValueError, Exception, BaseException, object])

In [ ]:
class CustomStringException(BaseException):
    pass

In [ ]:
a = '1234'

try:
    a[0] = '10'

except TypeError as e:
    raise CustomStringException(e)

else:
    print('successfully modified', a)

CustomStringException: 'str' object does not support item assignment

In [ ]:
a = ['1234', 2, 3]

try:
    a[0] = '10'

except TypeError as e:
    raise CustomStringException(e)

else:
    print(f'successfully modified {a=}')

successfully modified a=['10', 2, 3]


In [ ]:
raise CustomStringException('some message')

NameError: name 'CustomStringException' is not defined

In [ ]:
a = (0, 1)
try:
  a[0] = 1
except CustomStringException:
  pass

In [9]:
class A(Exception):
    pass

class B(A):
    pass

class C(B):
    pass

for cls in [C, B, A]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()

    except A:
        print('A')

    except B:
        print('B')

    except C:
        print('C')


raised C
A
raised B
A
raised A
A


In [11]:
for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()

    except B:
        print('B')

    except C:
        print('C')

    except A:
        print('A')



raised C
B
raised A
A
raised B
B


In [ ]:
for cls in [C, A, B]:
    try:
        print(f'raised {cls.__name__}')
        raise cls()

    except C:
        print('C')

    except B:
        print('B')

    except A:
        print('A')



raised C
C
raised A
A
raised B
B


Родительское исключение будет перехватывать исключения всех своих наследников

## class as a decorator

Экземпляры класса могут быть callable. А если функция может быть декоратором, то что мешает тогда классу? Должен быть определен magic метод `__call__`

In [ ]:
class ClassDecorator:
    def __init__(self, func):
        print("init")
        self.entry_str = '[decorating inner func]'
        self.function = func
        self.close_str = '[letting the song flow]'


    def __call__(self):
        print("call")
        print(self.entry_str)
        self.function()
        print(self.close_str)



@ClassDecorator
def function():
    print('Vanderlyle crybaby cry...')

init


In [ ]:
function()

call
[decorating inner func]
Vanderlyle crybaby cry...
[letting the song flow]


Как и в декораторах-функциях, мы тоже можем возвращать значения (а не только использовать print) и делать параметры

Сделаем таймер

In [ ]:
import time

class Timer:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        t_start = time.time()
        res = self.func(*args)
        total_time = time.time() - t_start
        print(f'{self.func.__name__}({args}) -> {res} executed in {total_time:.2f}s')
        return res

In [ ]:
@Timer
def slow_func(seconds_to_sleep):
    time.sleep(seconds_to_sleep)
    return 73

In [ ]:
slow_func(3)

slow_func((3,)) -> 73 executed in 3.00s


73

## class as decorated

Класс может быть тоже декорирован функцией или классом

In [ ]:
def decorator_function(target):

    def decorator_init(self, *args):
        print("DECORATED")

    print('I changed __init__ in class')
    target.__init__ = decorator_init

    return target


class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")


t = Target('one', 1)
t.__dict__

Target running


{'arg_list': ('one', 1)}

In [ ]:
@decorator_function
class Target:

    def __init__(self, *args):
        self.arg_list = args
        print("Target running")

I changed __init__ in class


In [ ]:
dt = Target('one', 1)
dt.__dict__

DECORATED


{}

In [ ]:
# А можете помочь разобраться как работает такой код, как декоратор понимает, что внутренний аргумент, это часть основной функции?:
# декоратор
def param_transfer(fn):
   def wrapper(arg):
       print("Run function: " + str(fn.__name__) + "(), with param: " + str(arg))
       fn(arg)
   return wrapper

# функция с декоратором
@param_transfer
def print_sqrt(num):
   print(num**0.5)

In [ ]:
print_sqrt(5)

Run function: print_sqrt(), with param: 5
2.23606797749979
